# Install All the Required Packages

In [6]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 datasets

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fmeval 1.0.3 requires transformers<5.0.0,>=4.36.0, but you have transformers 4.31.0 which is incompatible.
selective-context 0.1.4 requires click==8.0.4, but you have click 8.1.7 which is incompatible.
selective-context 0.1.4 requires spacy==3.2.0, but you have spacy 3.7.4 which is incompatible.
sentence-transformers 3.0.1 requires transformers<5.0.0,>=4.34.0, but you have transformers 4.31.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# pip install tensorboardX

# Import all the Required Libraries

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the Curated LIMA Dataset

In [2]:
dataset = load_dataset("Arnab13/LIMA-Generated-Instruct-self-curated")

HF google storage unreachable. Downloading and preparing it from source
Generating train split: 100%|██████████| 148/148 [00:00<00:00, 4441.62 examples/s]


In [3]:
def filter_scores(example):  
    return example['Score'] is not None and float(example['Score']) >= 3  

In [4]:
curated_dataset = dataset.filter(filter_scores)

Filter: 100%|██████████| 148/148 [00:00<00:00, 2631.90 examples/s]


In [5]:
# Function to concatenate the columns  
def concatenate_columns(example):  
    example['text'] = f"<s>[INST] {example['generated_instruction']} [/INST] {example['response']} </s>"  
    return example  

In [15]:
curated_dataset['train']

Dataset({
    features: ['generated_instruction', 'response', 'Score', 'generated_response', 'text'],
    num_rows: 148
})

In [6]:
# Apply the function to the dataset  
curated_dataset = dataset.map(concatenate_columns)

Map: 100%|██████████| 148/148 [00:00<00:00, 1911.83 examples/s]


# Setting up parameters for Fine-Tuning

QLoRA will use a rank of 64 with a scaling parameter of 16. We’ll load the Llama 2 model directly in 4-bit precision using the NF4 type and train it for one epoch

In [7]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-chat-hf"
# Fine-tuned model name
new_model = "Llama2-7b-Instruct"

In [8]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [9]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [10]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

In [11]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Fine-tuning

1. We load the dataset we defined. Here, our dataset is already preprocessed but, usually, this is where you would reformat the prompt, filter out bad text, combine multiple datasets, etc.

2. Then, we’re configuring bitsandbytes for 4-bit quantization.

3. Next, we're loading the Llama 2 model in 4-bit precision on a GPU with the corresponding tokenizer.

4. Finally, we're loading configurations for QLoRA, regular training parameters, and passing everything to the SFTTrainer. The training can finally start!

In [16]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=curated_dataset['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Your GPU supports bfloat16: accelerate training with bf16=True


Map: 100%|██████████| 148/148 [00:00<00:00, 1925.75 examples/s]


In [17]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,2.139700


TrainOutput(global_step=37, training_loss=2.1024552938100456, metrics={'train_runtime': 33.6293, 'train_samples_per_second': 4.401, 'train_steps_per_second': 1.1, 'total_flos': 1220958855168000.0, 'train_loss': 2.1024552938100456, 'epoch': 1.0})

In [18]:
# Save trained model
trainer.model.save_pretrained(new_model)

# Inference

In [19]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [21]:
for i in range(5):
    prompt = curated_dataset['train'][i]['generated_instruction']
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    print('Instruction: '+prompt+'\n')
    print('Response: ' + result[0]['generated_text'].split('[/INST] ')[-1])
    print("-" * 80)  

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/jovyan/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Instruction: I want to create a plot using Python. What is the best library to use?

Response:  There are several libraries available in Python for creating plots, depending on your needs and preferences. Here are some of the most popular ones:

1. Matplotlib: Matplotlib is one of the most popular libraries for creating plots in Python. It offers a wide range of features, including line plots, scatter plots, histograms, and more. Matplotlib is also highly customizable, making it easy to create complex and detailed plots.
2. Seaborn: Seaborn is a visualization library built on top of Matplotlib. It provides a higher-level interface for creating plots, making it easier to create beautiful and informative plots with a few lines of code. Seaborn also includes a range of built-in themes and visualization options, making it a great choice for creating plots with a professional look.
3.
--------------------------------------------------------------------------------
Instruction: How to open t

# Upload final self-aligned model to Huggingface hub

In [25]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

We need to merge the weights from LoRA with the base model.

In [26]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.96s/it]


## Push Model to Hugging Face Hub

Our weights are merged and we reloaded the tokenizer. We can now push everything to the Hugging Face Hub to save our model.

In [27]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [28]:
model.push_to_hub("Llama2-7b-Humpback", check_pr=True)
tokenizer.push_to_hub("Llama2-7b-Humpback",check_pr=True)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]
pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]
pytorch_model-00002-of-00002.bin:   0%|          | 8.19k/3.50G [00:00<34:01:16, 28.6kB/s]
pytorch_model-00002-of-00002.bin:   0%|          | 360k/3.50G [00:00<51:53, 1.12MB/s]    
pytorch_model-00002-of-00002.bin:   0%|          | 5.37M/3.50G [00:00<03:46, 15.4MB/s]
pytorch_model-00002-of-00002.bin:   0%|          | 12.9M/3.50G [00:01<06:26, 9.03MB/s]

pytorch_model-00001-of-00002.bin:   0%|          | 8.19k/9.98G [00:01<531:23:00, 5.22kB/s]

pytorch_model-00001-of-00002.bin:   0%|          | 12.6M/9.98G [00:01<15:49, 10.5MB/s]    
pytorch_model-00002-of-00002.bin:   0%|          | 16.0M/3.50G [00:04<17:48, 3.26MB/s]
pytorch_model-00002-of-00002.bin:   1%|          | 20.2M/3.50G [00:04<12:08, 4.78MB/s]
pytorch_model-00002-of-00002.bin:   1%|          | 29.5M/3.50G [00:05<10:

CommitInfo(commit_url='https://huggingface.co/Arnab13/Llama2-7b-Humpback/commit/88cfacd50629706612f0385f82addf03a8ca01e8', commit_message='Upload tokenizer', commit_description='', oid='88cfacd50629706612f0385f82addf03a8ca01e8', pr_url=None, pr_revision=None, pr_num=None)

You can now use this model for inference by loading it like any other Llama 2 model from the Hub.